In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install sinling gensim pyLDAvis pandas


In [3]:
import os
import pandas as pd
from sinling import SinhalaTokenizer, SinhalaStemmer
from gensim import corpora
from gensim.models import LdaModel
import pyLDAvis.gensim_models as vis
import pyLDAvis

# Adjust these paths if your files live in a subfolder
TRAIN_CSV = '/content/drive/MyDrive/NLP-Assignment/NSINA-Media-train.csv'
TEST_CSV  = '/content/drive/MyDrive/NLP-Assignment/NSINA-Media-test.csv'


In [4]:
df_train = pd.read_csv(TRAIN_CSV)
df_test  = pd.read_csv(TEST_CSV)

print("Train columns:", df_train.columns)
print("Test columns: ", df_test.columns)
df_train.head()


Train columns: Index(['News Content', 'Source'], dtype='object')
Test columns:  Index(['News Content', 'Source'], dtype='object')


,News Content,Source
0,"වැල්ලව, ලුණුකඩවැල්ල ප්‍රදේශයේදී ත්‍රීරෝද රථයක්...",Adaderana
1,රුපියල් මිලියන 15 ක වියදමින් කෑගල්ල දිස්ත්‍රික...,https://sinhala.news.lk
2,තාරුණ්‍යයට හෙටක් සංවිධානයේ මෙහෙයවීමෙන් ක්‍රියා...,https://sinhala.news.lk
3,"(ප්‍රසන්න පත්මසිරි)බිබිල මුදියල ගම්මානයේ, මාර්...",www.lankadeepa.lk
4,"මේ ව්‍යසනය ජනතාව කැඳවාගත් එකක් නෙවෙයි. මේ අමන,...",Lankatruth


In [12]:
#Preprocess using sinling
tok  = SinhalaTokenizer()
stem = SinhalaStemmer()

df_train['content'] = df_train['News Content'].astype(str)
df_test ['content'] = df_test ['News Content'].astype(str)

SINHALA_STOPWORDS = set([
    "අ", "ආ", "ඉ", "ඊ", "උ", "ඌ", "ඍ", "ඎ", "එ", "ඒ", "ඔ", "ඕ", "ඓ", "ඖ", "අං", "අඃ","ක", "ඛ", "ග", "ඝ", "ඞ",
    "ච", "ඡ", "ජ", "ඣ", "ඤ",
    "ට", "ඨ", "ඩ", "ඪ", "ණ",
    "ත", "ථ", "ද", "ධ", "න",
    "ප", "ඵ", "බ", "භ", "ම",
    "ය", "ර", "ල", "ව",
    "ශ", "ෂ", "ස", "හ", "ළ", "ෆ"
])

def preprocess(doc):
    tokens = []
    for sent in tok.split_sentences(str(doc)):
        for w in tok.tokenize(sent):
            if len(w) > 1 and w not in SINHALA_STOPWORDS:
                tokens.append(stem.stem(w))
    return tokens

# Apply to both sets (you’ll train on df_train, later infer on df_test)
df_train['tokens'] = df_train['content'].map(preprocess)
df_test ['tokens'] = df_test ['content'].map(preprocess)

print("Train tokens, first doc:", df_train['tokens'].iloc[0])
print("Type of each token:", [type(t) for t in df_train['tokens'].iloc[0][:5]])

Train tokens, first doc: [('වැල්ල', 'ව'), ('ලුණුකඩවැල්', 'ල'), ('ප්\u200dරදේශ', 'යේදී'), ('ත්\u200dරීරෝ', 'ද'), ('රථ', 'යක්'), ('ජ', 'ල'), ('පහර', 'කට'), ('හස', 'ුව'), ('ගසාගෙ', 'න'), ('ගොස', '්'), ('අතුරුදන්', 'ව'), ('තිබ', 'ේ'), ('ලුණුකඩවැල්', 'ල'), ('දැදුර', 'ු'), ('ඔ', 'ය'), ('සපත්ත', 'ු'), ('පාළ', 'ම'), ('මත', 'ින්'), ('ත්\u200dරීරෝ', 'ද'), ('රථ', 'ය'), ('ගම', 'න්'), ('කර', 'මින්'), ('තිබියද', 'ී'), ('ජ', 'ල'), ('පහර', 'ට'), ('හස', 'ුව'), ('ඇ', 'ති'), ('බ', 'ව'), ('පොලීස', 'ිය'), ('පැවසුවේ', 'ය'), ('කාන්තා', 'වක්'), ('ඇතුළ', 'ු'), ('සි', 'ව්'), ('දෙන', 'ෙකු'), ('ත්\u200dරීරෝ', 'ද'), ('රථ', 'යේ'), ('ගම', 'න්'), ('කර', ''), ('ඇ', 'ති'), ('අතර', ''), ('කාන්තා', 'වක'), ('හ', 'ා'), ('ත', 'වත්'), ('පුද්ගල', 'යෙකු'), ('දිවි', ''), ('බේරාගෙ', 'න'), ('තිබ', 'ේ'), ('පොලීස', 'ිය'), ('හ', 'ා'), ('ප්\u200dරදේශවාසී', 'න්'), ('එක්', 'ව'), ('අතුරුද', 'න්'), ('ව', 'ූ'), ('ත්\u200dරීරෝ', 'ද'), ('රථ', 'ය'), ('හ', 'ා'), ('අන', 'ෙක්'), ('පුද්ගලය', 'ින්'), ('දෙදෙ', 'නා'), ('සෙව', 'ීමේ'), ('කටයුත', 'ු')

In [13]:
from itertools import chain
from gensim import corpora

def flatten_and_extract(doc):
    flat = list(chain.from_iterable(doc)) if any(isinstance(el, list) for el in doc) else doc
    return [ token if isinstance(token, str) else token[0] for token in flat ]

df_train['clean_tokens'] = df_train['tokens'].apply(flatten_and_extract)

In [14]:
texts      = df_train['clean_tokens'].tolist()
dictionary = corpora.Dictionary(texts)
print("Number of unique tokens:", len(dictionary))
print("First 10 token→id mappings:", list(dictionary.token2id.items())[:10])

Number of unique tokens: 316762
First 10 token→id mappings: [('අතර', 0), ('අතුරුද', 1), ('අතුරුදන්', 2), ('අන', 3), ('ඇ', 4), ('ඇතුළ', 5), ('එක්', 6), ('ඔ', 7), ('කටයුත', 8), ('කර', 9)]


In [15]:
dictionary.filter_extremes(no_below=10, no_above=0.3, keep_n=10000)
# Check again
print("After filtering, vocab size:", len(dictionary))

After filtering, vocab size: 10000


In [16]:
# Spot-check a token that used to be common:
for tok_str in ["හ", "කොවිඩ්", "ක්‍රීඩා"]:
    print(f" '{tok_str}' in dictionary?", tok_str in dictionary.token2id)
train_corpus = [dictionary.doc2bow(text) for text in texts]
print("First doc bow (token_id, count):", train_corpus[0][:10])
print("Unique tokens in doc 0:", len(train_corpus[0]))

 'හ' in dictionary? False
 'කොවිඩ්' in dictionary? False
 'ක්‍රීඩා' in dictionary? True
First doc bow (token_id, count): [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 2), (7, 1), (8, 1), (9, 2)]
Unique tokens in doc 0: 33


In [17]:
from gensim.models import LdaModel

lda = LdaModel(
    corpus=train_corpus,
    id2word=dictionary,
    num_topics=6,
    passes=15,
    random_state=42
)

# Print top words per topic
for idx, topic in lda.show_topics(num_words=8, formatted=False):
    words = ", ".join(w for w,_ in topic)
    print(f"Topic {idx:>2d}: {words}")


Topic  0: ම, කිය, අපි, අප, වෙ, තම, නැහැ, ඔ
Topic  1: තිබෙ, මුදල, රජ, රුපියල, තුළ, හැ, පමණ, ආර්ථික
Topic  2: රට, තුළ, අප, යුත, ආණ්ඩ, දේශපාල, හැ, ඔව
Topic  3: ඉන්දියා, දෙමළ, ශ්‍ර, අපි, ම, කඳුකර, ස, කිය
Topic  4: පොලිස, අත්අඩංගු, ප්‍රදේශ, ගෙ, මරණ, අධිකරණ, පරීක්ෂණ, අනතුර
Topic  5: ශ්‍ර, ලං, ජනාධිප, කණ්ඩාය, පාර්ලිමේන්ත, ජාතික, තරග, ප්‍රකාශ


In [22]:
HUMAN_LABELS = {
    0: "සාමාන්‍ය පුවත්",
    1: "ආර්ථික පුවත්",
    2: "දේශපාලන පුවත්",
    3: "විදේශ පුවත්",
    4: "අපරාධ පුවත්",
    5: "ක්‍රීඩා පුවත්",
}

In [20]:
from itertools import chain

def flatten_and_extract(doc):
    """
    - If `doc` is a list of lists, flattens one level
    - Then for each element:
       • if it's a (word, tag) tuple → take word
       • else assume it’s already a string → keep it
    """
    # flatten one level if needed
    flat = list(chain.from_iterable(doc)) if any(isinstance(el, list) for el in doc) else doc
    # extract just the word text
    return [ token if isinstance(token, str) else token[0] for token in flat ]

def infer_topic_py(text, top_n=3):
    # 1. preprocess → might still give you tuples if you ever switch to a POS‐tagger
    raw = preprocess(text)
    # 2. clean it into pure strings
    toks = flatten_and_extract(raw)
    # 3. bag of words
    bow  = dictionary.doc2bow(toks)
    # 4. infer and sort
    topics = sorted(lda.get_document_topics(bow), key=lambda x: -x[1])
    return topics[:top_n]

# === Test it ===
sample = "අද කොළඹදී ක්‍රීඩා උළෙලක් පැවැත්වෙයි"
print("Raw preprocess:", preprocess(sample))
print("Flattened tokens:", flatten_and_extract(preprocess(sample)))
print("Inference result:", infer_topic_py(sample))


Raw preprocess: [('අ', 'ද'), ('කොළඹද', 'ී'), ('ක්\u200dරීඩ', 'ා'), ('උළෙල', 'ක්'), ('පැවැත්වෙ', 'යි')]
Flattened tokens: ['අ', 'කොළඹද', 'ක්\u200dරීඩ', 'උළෙල', 'පැවැත්වෙ']
Inference result: [(5, 0.8332567), (0, 0.033369098), (2, 0.03334643)]


In [21]:
dictionary.save("/content/drive/MyDrive/NLP-Assignment/Outputs/dictionary.dict")
lda.save("/content/drive/MyDrive/NLP-Assignment/Outputs/model.lda")


In [23]:
with open("/content/drive/MyDrive/NLP-Assignment/Outputs/labels.json", "w", encoding="utf-8") as f:
    json.dump(HUMAN_LABELS, f, ensure_ascii=False, indent=2)
